# This notebook calculates the winrate of a team

In [49]:
import pandas as pd
import numpy as np
pd.options.display.max_columns = None

In [129]:
# We start of by importing the pivot tables for synergy-winrate and counter-winrate
synergy_pivot = pd.read_pickle("../data/synergy_pivot.pkl")
counter_pivot = pd.read_pickle("../data/counter_pivot.pkl")
match_data = pd.read_pickle("../data/df_matches_grouped.pkl")

In [8]:
synergy_pivot

hero_2,1,2,3,4,5,6,7,8,9,10,...,111,112,113,114,119,120,121,126,128,129
hero_1,,,,,,,,,,,,,,,,,,,,,
1,NaN,0.55,0.60,0.56,0.57,0.53,0.55,0.53,0.55,0.52,...,0.52,0.56,0.62,0.54,0.56,0.55,0.52,0.53,0.53,0.49
2,0.55,NaN,0.54,0.57,0.54,0.54,0.48,0.51,0.54,0.49,...,0.50,0.50,0.58,0.52,0.51,0.48,0.45,0.52,0.55,0.47
3,0.60,0.54,NaN,0.55,0.47,0.58,0.55,0.54,0.56,0.51,...,0.38,0.49,0.49,0.49,0.62,0.54,0.49,0.56,0.56,0.50
4,0.56,0.57,0.55,NaN,0.57,0.50,0.51,0.54,0.56,0.48,...,0.55,0.57,0.43,0.44,0.56,0.50,0.51,0.55,0.47,0.49
5,0.57,0.54,0.47,0.57,NaN,0.52,0.47,0.51,0.54,0.49,...,0.51,0.52,0.52,0.50,0.48,0.50,0.46,0.51,0.46,0.46
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120,0.55,0.48,0.54,0.50,0.50,0.47,0.49,0.49,0.50,0.45,...,0.49,0.47,0.42,0.43,0.48,NaN,0.49,0.48,0.50,0.41
121,0.52,0.45,0.49,0.51,0.46,0.44,0.47,0.45,0.44,0.46,...,0.37,0.42,0.53,0.44,0.46,0.49,NaN,0.47,0.45,0.40
126,0.53,0.52,0.56,0.55,0.51,0.51,0.50,0.51,0.54,0.44,...,0.41,0.49,0.51,0.48,0.49,0.48,0.47,NaN,0.50,0.50


### 1. We start off by writing a function to lookup the value of a pair-winrate

In [11]:
def winrate_selector_synergy(hero_pair_tuple):
    
    # Locate the hero chosen on the team
    hero_matchup_rows = synergy_pivot.loc[hero_pair_tuple[0]]
    
    # We select the given hero through the index of the list
    winrate = hero_matchup_rows[hero_pair_tuple[1]]
    
    # We return the winrate
    return winrate

In [12]:
def winrate_selector_counter(hero_pair_tuple):
    
    # Locate the hero chosen on the team
    hero_matchup_rows = counter_pivot.loc[hero_pair_tuple[0]]
    
    # We select the given hero through the index of the list
    winrate = hero_matchup_rows[hero_pair_tuple[1]]
    
    # We return the winrate
    return winrate

### 2. We make a function that calculates the winrate of the tuples in the match

In [13]:
match_data

,match_id,match_seq_num,radiant_win,start_time,duration,avg_mmr,radiant_1,radiant_2,radiant_3,radiant_4,radiant_5,dire_1,dire_2,dire_3,dire_4,dire_5
0,5510478612,4620417806,True,1594586690,2721,3174,7,70,8,30,105,119,73,90,31,18
1,5510478615,4620399961,False,1594586691,1155,3623,86,49,84,75,57,40,120,30,1,60
2,5510478700,4620425526,False,1594586691,3281,3243,108,71,8,26,47,59,74,30,21,14
3,5510478702,4620421211,True,1594586695,2988,2405,34,8,10,86,53,48,18,25,85,75
4,5510478708,4620414900,True,1594586700,2277,4464,67,22,37,107,14,74,7,84,45,44
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,5513436913,4623081152,False,1594771169,2254,3031,69,63,84,7,44,2,45,22,27,73
99996,5513436918,4623070259,True,1594771180,1688,5019,86,53,1,19,84,104,39,113,26,54
99997,5513437003,4623069709,False,1594771184,1790,864,15,108,21,30,53,126,104,110,4,84
99998,5513437208,4623072527,True,1594771189,1928,4327,27,93,6,84,81,10,67,120,101,104


In [126]:
def synergy_winrate_calculator(match_data, player_slots):
    for slot1 in player_slots:
        for slot2 in player_slots:
            if slot1 != slot2:
                series = pd.DataFrame(zip(match_data[slot1].values,match_data[slot2].values))
                winrate = series.apply(winrate_selector_synergy, axis=1)
                match_data[f"syn-{slot1}-{slot2}"] = winrate
                print("Synergy ",slot1,slot2)
    return match_data

def counter_winrate_calculator(match_data, player_slots_team, player_slots_opponent):
    for slot1 in player_slots_team:
        for slot2 in player_slots_opponent:
            series = pd.DataFrame(zip(match_data[slot1].values,match_data[slot2].values))
            winrate = series.apply(winrate_selector_counter, axis=1)
            match_data[f"coun-{slot1}-{slot2}"] = winrate
            print("Counter ",slot1,slot2)
    return match_data

In [124]:
def winrate_calculator(match_data):
    
    # Declare variables
    radiant_slots = ["radiant_1",
                     "radiant_2",
                     "radiant_3",
                     "radiant_4",
                     "radiant_5"]
    
    dire_slots = ["dire_1",
                  "dire_2",
                  "dire_3",
                  "dire_4",
                  "dire_5"]
    
    columns = ["match_id",
               "match_seq_num",
               "radiant_win",
               "start_time",
               "duration",
               "avg_mmr",
               "radiant_1",
               "radiant_2",
               "radiant_3",
               "radiant_4",
               "radiant_5",
               "dire_1",
               "dire_2",
               "dire_3",
               "dire_4",
               "dire_5",
               "radiant_synergy_wr",
               "radiant_counter_wr",
               "dire_synergy_wr",
               "dire_counter_wr"]
    
    winrate_radiant_syn_list = []
    winrate_dire_syn_list = []
    winrate_radiant_coun_list = []
    winrate_dire_coun_list = []
    radiant_winrate = 0
    dire_winrate = 0
    copy_match_data = match_data
    
    # Radiant winrate based on synergy and counter
    radiant_synergy_winrate_df = synergy_winrate_calculator(copy_match_data, radiant_slots)
    radiant_counter_winrate_df = counter_winrate_calculator(copy_match_data, radiant_slots, dire_slots)
    syn_columns = radiant_synergy_winrate_df.columns[16:36].tolist()
    coun_columns = radiant_counter_winrate_df.columns[36:61].tolist()
    match_data["radiant_synergy_wr"] = copy_match_data[syn_columns].apply(np.mean, axis=1)
    match_data["radiant_counter_wr"] = copy_match_data[coun_columns].apply(np.mean, axis=1)
    
    # Dire winrate based on synergy and counter
    dire_synergy_winrate_df = synergy_winrate_calculator(copy_match_data, dire_slots)
    syn_columns = dire_synergy_winrate_df.columns[63:83].tolist()
    match_data["dire_synergy_wr"] = copy_match_data[syn_columns].apply(np.mean, axis=1)
    match_data["dire_counter_wr"] = 1 - match_data["radiant_counter_wr"]
    
    return match_data[columns]

In [125]:
match_data_pred = winrate_calculator(match_data)

Synergy  radiant_1 radiant_2


KeyboardInterrupt: 

In [109]:
match_data_pred

,match_id,match_seq_num,radiant_win,start_time,duration,avg_mmr,radiant_1,radiant_2,radiant_3,radiant_4,radiant_5,dire_1,dire_2,dire_3,dire_4,dire_5,radiant_synergy_wr,radiant_counter_wr,dire_synergy_wr,dire_counter_wr
0,5510478612,4620417806,True,1594586690,2721,3174,7,70,8,30,105,119,73,90,31,18,0.503,0.5076,0.460,0.4924
1,5510478615,4620399961,False,1594586691,1155,3623,86,49,84,75,57,40,120,30,1,60,0.471,0.4716,0.541,0.5284
2,5510478700,4620425526,False,1594586691,3281,3243,108,71,8,26,47,59,74,30,21,14,0.512,0.5088,0.481,0.4912
3,5510478702,4620421211,True,1594586695,2988,2405,34,8,10,86,53,48,18,25,85,75,0.450,0.4860,0.476,0.5140
4,5510478708,4620414900,True,1594586700,2277,4464,67,22,37,107,14,74,7,84,45,44,0.547,0.5300,0.493,0.4700
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,5510502107,4620438087,True,1594588341,2615,3434,86,7,70,23,75,1,14,39,80,5,0.478,0.4748,0.546,0.5252
996,5510502111,4620449399,False,1594588352,3678,1387,39,73,41,26,4,68,12,7,94,22,0.499,0.4900,0.524,0.5100
997,5510502113,4620435720,True,1594588354,2364,2755,42,5,100,99,39,8,74,11,34,47,0.520,0.5256,0.456,0.4744
998,5510502114,4620435304,False,1594588357,2481,2171,35,114,45,39,42,1,53,100,120,3,0.518,0.5008,0.511,0.4992


**We now create a predictor function to account for the different matchups**

In [118]:
def win_predictor(match_data_pred):
    match_data_pred["counter_radiant_win"] = np.where(match_data_pred["radiant_counter_wr"] >= 0.5, True, False)
    match_data_pred["synergy_radiant_win"] = np.where(match_data_pred["radiant_synergy_wr"] >= match_data_pred["dire_synergy_wr"], True, False)
    match_data_pred["synergy_precision"] = np.where(match_data_pred["radiant_win"] == match_data_pred["synergy_radiant_win"], True, False)
    match_data_pred["counter_precision"] = np.where(match_data_pred["radiant_win"] == match_data_pred["counter_radiant_win"], True, False)
    
    synergy_counter = match_data_pred["synergy_precision"].value_counts()
    counter_counter = match_data_pred["counter_precision"].value_counts()
    synergy_precision = synergy_counter / len(match_data_pred["synergy_precision"])
    counter_precision = counter_counter / len(match_data_pred["synergy_precision"])
    
    return match_data_pred

In [119]:
win_predictor(match_data_pred)

,match_id,match_seq_num,radiant_win,start_time,duration,avg_mmr,radiant_1,radiant_2,radiant_3,radiant_4,radiant_5,dire_1,dire_2,dire_3,dire_4,dire_5,radiant_synergy_wr,radiant_counter_wr,dire_synergy_wr,dire_counter_wr,counter_radiant_win,synergy_radiant_win,synergy_precision,counter_precision
0,5510478612,4620417806,True,1594586690,2721,3174,7,70,8,30,105,119,73,90,31,18,0.503,0.5076,0.460,0.4924,True,True,True,True
1,5510478615,4620399961,False,1594586691,1155,3623,86,49,84,75,57,40,120,30,1,60,0.471,0.4716,0.541,0.5284,False,False,True,True
2,5510478700,4620425526,False,1594586691,3281,3243,108,71,8,26,47,59,74,30,21,14,0.512,0.5088,0.481,0.4912,True,True,False,False
3,5510478702,4620421211,True,1594586695,2988,2405,34,8,10,86,53,48,18,25,85,75,0.450,0.4860,0.476,0.5140,False,False,False,False
4,5510478708,4620414900,True,1594586700,2277,4464,67,22,37,107,14,74,7,84,45,44,0.547,0.5300,0.493,0.4700,True,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,5510502107,4620438087,True,1594588341,2615,3434,86,7,70,23,75,1,14,39,80,5,0.478,0.4748,0.546,0.5252,False,False,False,False
996,5510502111,4620449399,False,1594588352,3678,1387,39,73,41,26,4,68,12,7,94,22,0.499,0.4900,0.524,0.5100,False,False,True,True
997,5510502113,4620435720,True,1594588354,2364,2755,42,5,100,99,39,8,74,11,34,47,0.520,0.5256,0.456,0.4744,True,True,True,True
998,5510502114,4620435304,False,1594588357,2481,2171,35,114,45,39,42,1,53,100,120,3,0.518,0.5008,0.511,0.4992,True,True,False,False


In [130]:
match_data

,match_id,match_seq_num,radiant_win,start_time,duration,avg_mmr,radiant_1,radiant_2,radiant_3,radiant_4,radiant_5,dire_1,dire_2,dire_3,dire_4,dire_5
0,5510478612,4620417806,True,1594586690,2721,3174,7,70,8,30,105,119,73,90,31,18
1,5510478615,4620399961,False,1594586691,1155,3623,86,49,84,75,57,40,120,30,1,60
2,5510478700,4620425526,False,1594586691,3281,3243,108,71,8,26,47,59,74,30,21,14
3,5510478702,4620421211,True,1594586695,2988,2405,34,8,10,86,53,48,18,25,85,75
4,5510478708,4620414900,True,1594586700,2277,4464,67,22,37,107,14,74,7,84,45,44
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,5513436913,4623081152,False,1594771169,2254,3031,69,63,84,7,44,2,45,22,27,73
99996,5513436918,4623070259,True,1594771180,1688,5019,86,53,1,19,84,104,39,113,26,54
99997,5513437003,4623069709,False,1594771184,1790,864,15,108,21,30,53,126,104,110,4,84
99998,5513437208,4623072527,True,1594771189,1928,4327,27,93,6,84,81,10,67,120,101,104


In [131]:
synergy_pivot

hero_2,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,119,120,121,126,128,129
hero_1,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,NaN,0.55,0.60,0.56,0.57,0.53,0.55,0.53,0.55,0.52,0.51,0.52,0.51,0.55,0.46,0.56,0.51,0.56,0.51,0.57,0.56,0.59,0.55,0.55,0.53,0.55,0.54,0.52,0.56,0.58,0.60,0.56,0.57,0.57,0.58,0.57,0.58,0.56,0.59,0.56,0.59,0.47,0.48,0.55,0.49,0.57,0.61,0.55,0.55,0.58,0.45,0.48,0.52,0.50,0.50,0.54,0.48,0.50,0.55,0.62,0.52,0.55,0.59,0.48,0.60,0.59,0.57,0.48,0.54,0.56,0.45,0.50,0.56,0.53,0.50,0.56,0.48,0.47,0.59,0.54,0.59,0.57,0.58,0.58,0.50,0.55,0.56,0.50,0.54,0.52,0.52,0.50,0.52,0.48,0.54,0.51,0.52,0.55,0.51,0.55,0.59,0.55,0.56,0.57,0.52,0.52,0.60,0.43,0.58,0.52,0.56,0.62,0.54,0.56,0.55,0.52,0.53,0.53,0.49
2,0.55,NaN,0.54,0.57,0.54,0.54,0.48,0.51,0.54,0.49,0.49,0.52,0.48,0.51,0.52,0.58,0.44,0.46,0.40,0.56,0.50,0.59,0.53,0.48,0.49,0.51,0.44,0.50,0.53,0.51,0.51,0.56,0.50,0.54,0.47,0.54,0.38,0.51,0.52,0.50,0.56,0.49,0.50,0.54,0.46,0.52,0.57,0.59,0.53,0.57,0.55,0.47,0.51,0.48,0.56,0.53,0.40,0.53,0.42,0.54,0.50,0.53,0.55,0.46,0.42,0.56,0.58,0.56,0.56,0.51,0.50,0.48,0.52,0.49,0.44,0.52,0.56,0.49,0.59,0.52,0.50,0.50,0.55,0.56,0.47,0.53,0.48,0.50,0.50,0.52,0.49,0.52,0.54,0.52,0.50,0.38,0.45,0.39,0.49,0.53,0.55,0.54,0.52,0.53,0.49,0.48,0.63,0.48,0.57,0.50,0.50,0.58,0.52,0.51,0.48,0.45,0.52,0.55,0.47
3,0.60,0.54,NaN,0.55,0.47,0.58,0.55,0.54,0.56,0.51,0.51,0.48,0.48,0.55,0.48,0.51,0.49,0.51,0.47,0.46,0.56,0.57,0.53,0.58,0.47,0.57,0.58,0.65,0.55,0.50,0.54,0.45,0.56,0.55,0.55,0.61,0.56,0.56,0.53,0.53,0.55,0.51,0.48,0.56,0.56,0.60,0.55,0.53,0.53,0.59,0.41,0.51,0.50,0.56,0.61,0.56,0.50,0.57,0.57,0.64,0.51,0.52,0.61,0.40,0.55,0.62,0.59,0.51,0.54,0.43,0.52,0.55,0.53,0.42,0.39,0.68,0.56,0.40,0.63,0.46,0.55,0.56,0.55,0.48,0.47,0.50,0.64,0.51,0.54,0.40,0.57,0.53,0.53,0.56,0.54,0.56,0.43,0.48,0.51,0.54,0.59,0.61,0.58,0.61,0.52,0.51,0.57,0.52,0.58,0.38,0.49,0.49,0.49,0.62,0.54,0.49,0.56,0.56,0.50
4,0.56,0.57,0.55,NaN,0.57,0.50,0.51,0.54,0.56,0.48,0.47,0.51,0.48,0.52,0.52,0.49,0.46,0.52,0.46,0.51,0.50,0.56,0.52,0.49,0.50,0.53,0.53,0.54,0.55,0.54,0.50,0.52,0.47,0.55,0.49,0.57,0.51,0.55,0.57,0.51,0.61,0.52,0.49,0.55,0.44,0.47,0.56,0.46,0.55,0.63,0.41,0.49,0.47,0.54,0.47,0.52,0.45,0.52,0.56,0.55,0.54,0.53,0.54,0.47,0.50,0.63,0.60,0.53,0.56,0.53,0.41,0.52,0.51,0.49,0.39,0.28,0.45,0.43,0.52,0.45,0.52,0.50,0.58,0.48,0.51,0.46,0.51,0.50,0.53,0.46,0.43,0.55,0.57,0.56,0.59,0.51,0.47,0.59,0.47,0.53,0.57,0.54,0.59,0.56,0.45,0.42,0.57,0.49,0.54,0.55,0.57,0.43,0.44,0.56,0.50,0.51,0.55,0.47,0.49
5,0.57,0.54,0.47,0.57,NaN,0.52,0.47,0.51,0.54,0.49,0.51,0.48,0.49,0.52,0.49,0.48,0.44,0.54,0.42,0.48,0.51,0.58,0.49,0.48,0.48,0.49,0.50,0.53,0.51,0.52,0.53,0.56,0.48,0.51,0.56,0.49,0.52,0.54,0.50,0.49,0.56,0.44,0.50,0.50,0.50,0.49,0.52,0.48,0.52,0.54,0.34,0.48,0.49,0.46,0.59,0.49,0.42,0.46,0.58,0.53,0.53,0.52,0.49,0.53,0.48,0.57,0.50,0.52,0.54,0.52,0.39,0.52,0.49,0.54,0.41,0.44,0.43,0.51,0.54,0.47,0.47,0.53,0.58,0.54,0.46,0.52,0.54,0.40,0.51,0.36,0.47,0.52,0.53,0.50,0.52,0.46,0.51,0.51,0.46,0.53,0.49,0.39,0.53,0.52,0.52,0.52,0.54,0.40,0.52,0.51,0.52,0.52,0.50,0.48,0.50,0.46,0.51,0.46,0.46
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120,0.55,0.48,0.54,0.50,0.50,0.47,0.49

In [137]:
# hero_id_1 with hero_id_3
test_tuple = (1, 3)

# we find the synergy value of these heroes in the synergy pivot
hero_synergy_list = synergy_pivot.loc[test_tuple[0]]
hero_synergy_list
winrate = hero_synergy_list[test_tuple[1]]
winrate

0.6

0.6